# RPGF3 Projects Building on Base

###  Draft RPGF3 List ✨

In [1]:
from collections import Counter
import json
from nltk import ngrams
from nltk.tokenize import word_tokenize
import pandas as pd

from scripts.oso_db import execute_query
from scripts.eas_lists import OsoData, listify, create_eas_json

# Prep

In [16]:
# settings

DATA_DIR  = "data/RPGF3/"
DATA_PATH = DATA_DIR + "2023-11_06_RPGF3_cleaned_applicant_data.json"

LIST_NAME = "Base Builders"
LINK_BASE = "https://github.com/opensource-observer/insights/blob/main/notebooks/"
FILENAME  = "2023-11-08_BaseBuilders.ipynb"
LIST_LINK = LINK_BASE + FILENAME

LIST_DESC = " ".join([
                "This list is a shortlist of applications that are building on Base or claim other forms of",
                "impact on Base. It does not include any token allocations."
                ])
LIST_CATS = [
            #'COLLECTIVE_GOVERNANCE',
            'DEVELOPER_ECOSYSTEM', 
            'END_USER_EXPERIENCE_AND_ADOPTION', 
            'OP_STACK'
            ]

CSV_PATH = DATA_DIR + "lists/" + LIST_NAME.replace(" ","") + ".csv"

# Shortlist apps

In [8]:
def review_app(app):
    
    for contract_link in app["Contributions: Contracts"]:
        if "basescan" in contract_link:
            app["Base Evidence"] = "Onchain"
            return True

    for metric_link in app["Impact Metrics"]:
        if "basescan" in metric_link:
            app["Base Evidence"] = "Onchain"
            return True
    
    text = " ".join([app["Bio"], app["Contribution Description"], app["Impact Description"]]).lower()
    
    bigram_list = list(ngrams(text.split(), 2))
    if ('base', 'chain') in bigram_list or ('base', 'mainnet') in bigram_list:
        app["Base Evidence"] = "Narrative"
        return True
    
    tokens = word_tokenize(text, language='english', preserve_line=False)    
    word_counts = Counter(tokens)
    if word_counts["base"] > 2:
        app["Base Evidence"] = "Narrative"
        return True
    
    app["Base Evidence"] = None
    return False

with open(DATA_PATH) as f:
    clean_data = json.load(f)

base_shortlist = [app for app in clean_data if review_app(app)]    
len(base_shortlist)

57

# Export CSV for further analysis

In [24]:
cols = ['Project ID', 'Project Name', 'Slug: Primary', 'Bio', 'Website', 'Base Evidence', ]
df = pd.DataFrame(base_shortlist)[cols]

df['Agora Link'] = df['Project ID'].apply(lambda x: f"https://vote.optimism.io/retropgf/3/application/{x}")
df['West Link'] = df['Project ID'].apply(lambda x: f"https://round3.optimism.io/projects/{x}")
df['OSO Page'] = df['Slug: Primary'].apply(lambda x: f"https://opensource.observer/project/{x}" if x else None)

df.set_index('Project Name', inplace=True)
df.sort_index(inplace=True, key=lambda col: col.str.lower())
df.drop(columns=['Project ID', 'Slug: Primary'], inplace=True)

df.to_csv(CSV_PATH)
df

,Bio,Website,Base Evidence,Agora Link,West Link,OSO Page
Project Name,,,,,,
ACryptoS,Liquidity Manager & Optimizer - for V3 CL DEXs...,https://app.acryptos.com/,Narrative,https://vote.optimism.io/retropgf/3/applicatio...,https://round3.optimism.io/projects/0x5cc8dc85...,https://opensource.observer/project/acrypto-s
Aerodrome Finance,The central trading and liquidity marketplace ...,https://aerodrome.finance/,Narrative,https://vote.optimism.io/retropgf/3/applicatio...,https://round3.optimism.io/projects/0x8a98ca38...,https://opensource.observer/project/aerodrome-...
Alchemy,Building EVM account abstraction infrastructur...,https://www.alchemy.com/,Onchain,https://vote.optimism.io/retropgf/3/applicatio...,https://round3.optimism.io/projects/0xa771de85...,https://opensource.observer/project/alchemypla...
anotherblock,COLLECT – OWN – BELONG,https://anotherblock.io/,Narrative,https://vote.optimism.io/retropgf/3/applicatio...,https://round3.optimism.io/projects/0x709b9997...,https://opensource.observer/project/anotherblock
APAC DAO,The most extensive web3 B2B community in APAC ...,https://apacdao.net/,Narrative,https://vote.optimism.io/retropgf/3/applicatio...,https://round3.optimism.io/projects/0xd589501f...,None
API3 DAO,API3 has built an oracle node deployed to and ...,https://api3.org/,Narrative,https://vote.optimism.io/retropgf/3/applicatio...,https://round3.optimism.io/projects/0xaf653d36...,None
arabianhorses,A dashboard collections that built with BASE a...,https://flipsidecrypto.xyz/arabianhorses-jZtVN...,Onchain,https://vote.optimism.io/retropgf/3/applicatio...,https://round3.optimism.io/projects/0xf87b7f79...,None
Archly,Empowering projects and users to operate acros...,https://archly.fi,Onchain,https://vote.optimism.io/retropgf/3/applicatio...,https://round3.optimism.io/projects/0xee97ff73...,None
Base Punk,Base Punk is a Community-driven NFT project w...,https://www.basepunk.xyz/,Onchain,https://vote.optimism.io/retropgf/3/applicatio...,https://round3.optimism.io/projects/0xbe242c7f...,https://opensource.observer/project/basepunkorg
